# Tracing Basics

# Instead of OPENAI I've changed the code to use my Gemini API key

### Setup

Make sure you set your environment variables, including your OpenAI API key.

In [1]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [8]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="C:/Users/Lenovo/Documents/code/mat496/.env", override=True) #path to my .env file with given variables INCLUDING GEMINI API KEY


True

### Tracing with @traceable

The @traceable decorator is a simple way to log traces from the LangSmith Python SDK. Simply decorate any function with @traceable.

The decorator works by creating a run tree for you each time the function is called and inserting it within the current trace. The function inputs, name, and other information is then streamed to LangSmith. If the function raises an error or if it returns a response, that information is also added to the tree, and updates are patched to LangSmith so you can detect and diagnose sources of errors. This is all done on a background thread to avoid blocking your app's execution.

In [7]:
!pip install langchain-google-genai

  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached googleapis_common_protos-1.70.0-py3-none-any.whl.metadata (9.3 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 8.5 MB/s eta 0:00:00
Using cached googleapis_common_protos-1.70.0-py3-none-any.whl (294 kB)
   ---------------------------------------- 0.0/4.6 MB ? eta -:--:--
   ------------------------------------ --- 4.2/4.6 MB 20.7 MB/s eta 0:00:01
   ---------------------------------------- 4.6/4.6 MB 16.6 MB/s eta 0:00:00
Using cached proto_plus-1.26.1-py3-none-any.whl (50 kB)
Using cached rsa-4.9.1-py3-none-any.whl (34 kB)

   --- ------------------------------------  1/11 [rsa]
  Attempting uninstall: protobuf
   --- ------------------------

In [9]:
# Imports
import langchain_google_genai
from langsmith import traceable
from langchain_google_genai import ChatGoogleGenerativeAI 
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

# Configuration
MODEL_PROVIDER = "google"                       
MODEL_NAME = "gemini-2.5-flash"                
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the latest question in the conversation.
If you don't know the answer, just say that you don't know. Use bullet points and other formatting methods where needed.
You can make logical assumptions/deductions from provided context but do not assume any other information
"""

# Initialize

# The system prompt is passed here as per langchain specifications
llm = ChatGoogleGenerativeAI(
    model=MODEL_NAME, 
    temperature=0.3,
    max_tokens=1024,
    system_instruction=RAG_SYSTEM_PROMPT 
)


nest_asyncio.apply()
retriever = get_vector_db_retriever() # Assumed retriever initialization

# Traced Functions TODO DONE

@traceable(metadata={"vectordb": "sklearn"})
def retrieve_documents(question: str):
    """Retrieves documents using the vector store retriever."""
    return retriever.invoke(question)

@traceable
def generate_response(question: str, documents):
    """Prepares documents and calls the model for generation."""
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    
 
    messages = [
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    
    return call_gemini(messages)

@traceable(
    
    metadata={"model_name": MODEL_NAME, "model_provider": MODEL_PROVIDER}
)
def call_gemini(
    messages: List[dict]
) -> str:
    """Call Google's Gemini API via the pre-initialized LangChain Chat Model."""
    
    # We use the pre-initialized 'llm' object which holds the model and system prompt config
    response = llm.invoke(messages) 
    
    return response.content

@traceable
def langsmith_rag(question: str):
    """The full RAG pipeline entry point."""
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response


ModuleNotFoundError: No module named 'utils'

@traceable handles the RunTree lifecycle for you!

In [ ]:
question = "How can I trace with the @traceable decorator?"
ai_answer = langsmith_rag(question)
print(ai_answer)

##### Let's take a look in LangSmith!

### Adding Metadata

LangSmith supports sending arbitrary metadata along with traces.

Metadata is a collection of key-value pairs that can be attached to runs. Metadata can be used to store additional information about a run, such as the version of the application that generated the run, the environment in which the run was generated, or any other information that you want to associate with a run. Similar to tags, you can use metadata to filter runs in the LangSmith UI, and can be used to group runs together for analysis.

In [20]:
from langsmith import traceable

@traceable(
    # TODO: Add Metadata
    # metadata={"vectordb": "sklearn"}
)
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

@traceable(
    # TODO: Add Metadata
    # metadata={"model_name": MODEL_NAME, "model_provider": MODEL_PROVIDER}
)
def call_openai(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


In [ ]:
question = "How do I add Metadata to a Run with @traceable?"
ai_answer = langsmith_rag(question)
print(ai_answer)

You can also add metadata at runtime!

In [ ]:
question = "How do I add metadata at runtime?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"runtime_metadata": "foo"}})
print(ai_answer)

##### Let's take a look in LangSmith!